In [32]:
# Bow  Bag of Words  CountVectorizer
# 문서를 고정된 길이의 벡터로 변환
# 문서 - 단어행렬
# 장점 : 간단하고 빠름
# 단점 : 단어순서 손실, 희소성, 의미적 유사성 무시

# tf-idf  TfidfVectorizer 
# 모든 문서에서 자주 등장하는 단어의 영향을 줄이고 문서의 특이 단어를 수집

# multinomal Naive Bayes 확률 모델
# LogisticRegression : 다중클래스  회귀기반 분류

# RidgeClassifier : 회귀 기반 분류 L2규제(가중치 제곱의 값을 추가로 페널티 규제)

# N-gram : 단점 차원 폭발에 주의 (정규화/차원 축소 고려)

# Kolnpy Okt

In [33]:
import numpy as np
import pandas as pd
# scikit-learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
# 분류모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [34]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# data load
newsgroups_train = fetch_20newsgroups(subset='train',
                    remove=('headers', 'footers', 'quotes'),
                    categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                    remove=('headers', 'footers', 'quotes'),
                    categories=categories)


In [35]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories ]
    # 필터링
    data_filtered, target_filtered = [], []
    for text, label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)
            data_filtered.append(text); target_filtered.append(new_label)
    return data_filtered, target_filtered, categories

In [36]:
train_data, train_target, target_names = filter_categories(newsgroups_train, categories)
test_data, test_target, _ = filter_categories(newsgroups_test, categories)

In [37]:
# 헤더 푸터 인용문 제거
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].', '', text)

    return text

In [38]:
train_data = [ clean_text(t) for t in train_data ]
test_data = [ clean_text(t) for t in test_data ]

In [40]:
len(train_data), len(train_target), len(test_data), len(test_target)

(2034, 2034, 1353, 1353)

- 멀티노멀 나이즈베이즈
- 문서에 포함된 단어들의 출현 횟수를 기반으로 해서 그 문서가 어떤 주제에 속할지 확률적
- 스팸필터링, 뉴스기사 카테고리, 감성분석

- 베이즈정리 확률 이론 - 조건부 확률
- 단어 A가 나왔을때, 이 문서가 스팸 B 일 확률은 얼마

- 나이브 Naive : 순진한 가정
    - 가정 : 문서안의 모든 단어는 서로 독립적
    - 현실 : 스팸에 자주 나오는 단어들은 서로 독립적이지 않다
    - 실제 : 이러한 가정은 계산량을 빠르게 하고 단순하지만 정확도가 어느정도 나온다
- 멀티노멀 : 다항 분포
    - 의미 : 단어의 출현 횟수를 중요
    - 횟수를 세는 멀티노미얼 방식이 NLP 잘 맞는다
    - 모델은 단어의 빈도수 통계
    - 스팸메일 통계 spen
        - free : 150
        - money : 100
        - viagra : 50
        - report : 5
    - 정상메일 Ham 통계
        - report : 80
        - meeting : 60
        - free : 10
    - 이러한 통계를 바탕으로 이 카테고리에서 특정 단어가 나올 확률 (P'free'|스펨) 을 모두 계산
    - "Free mondy meeting"
        - 스팸?
            - 기본 스펨확률 x, 스펨일떄 free 가 나올 확률 x 스팸일떄 money 나올 확률 x 스팸일떄 meeting 나올 확률
        - 정상
            - 기본 정상확률 x 정상일떄...